In [2]:
import pandas as pd
import numpy as np
import itertools
import random
import ast
import matplotlib
import matplotlib.pyplot as plt
from scipy.stats import spearmanr, wilcoxon, friedmanchisquare, kendalltau, pearsonr
from scipy.spatial import distance
from fastDamerauLevenshtein import damerauLevenshtein

pd.set_option('display.max_rows', 500)

### Prepare Data

In [3]:
# read data

df = pd.read_csv('all_task_environments_2020-11-25.csv', header=0)

### Define functions for prediction

In [4]:
def predict_sequence(objects, coordinates, start_coordinates, c, k, dimension=[3,]):
    ''' Predicts sequence based on required objects, object coordinates, start coordinates of subject,
        parameters (c+k) and dimensionality.
        Input: Objects, object coordinates, start coordinates, c, k, dimension
        Output: Sequence of objects as str
    '''
    prediction = []
    possible_items = dict.fromkeys(objects, 0) # generate dict from object list
    coord_index = 0
    start_coords = start_coordinates
    coords = coordinates
    new_coords = {}
    new_start_coords = []
    
    if dimension[0] == 3: # no changes if 3D
        new_coords = coords
        new_start_coords = start_coords
        
    elif dimension[0] == 2: # 2D: remove obsolete coordinate
        if dimension[1] == 'xy':
            new_coords = {key: value[:-1] for key, value in coords.items()}
            new_start_coords = [x[:-1] for x in start_coords]
                
        elif dimension[1] == 'xz':
            new_start_coords = [[x[0], x[-1]] for x in start_coords]
            
            for key, value in coords.items():
                new_value = (value[0], value[-1])
                new_coords[key] = new_value
        
        elif dimension[1] == 'yz':
            new_coords = {key: value[1:] for key, value in coords.items()}
            new_start_coords = [x[1:] for x in start_coords]
                
    elif dimension[0] == 1: # 1D: choose appropriate coordinate
        if dimension[1] == 'x':
            new_coords = {key: value[0] for key, value in coords.items()}
            new_start_coords = [x[0] for x in start_coords]
        
        elif dimension[1] == 'y':
            new_coords = {key: value[1] for key, value in coords.items()}
            new_start_coords = [x[1] for x in start_coords]
        
        elif dimension[1] == 'z':
            new_coords = {key: value[2] for key, value in coords.items()}
            new_start_coords = [x[2] for x in start_coords]
    
    while bool(possible_items) == True: # while dict not empty
        for obj in possible_items.keys():
            possible_items[obj] = ((distance.euclidean(
                                new_start_coords[coord_index], 
                                new_coords[obj])
                                ) ** k[obj]) * c[obj]
        #print(possible_items)                      
        minval = min(possible_items.values())
        minval = [k for k, v in possible_items.items() if v == minval]
        minval = random.choice(minval) # choose prediction randomly if multiple items have same cost
        prediction.append(minval)
        del possible_items[minval]
        coord_index += 1
        
    return prediction

In [5]:
def get_median(objects, coordinates, start_coordinates, c, k, dimension, sequence,n=10):
    ''' Returns average edit distance (Damerau-Levenshtein) for 100 trials of sequence prediction.
    '''
    edit_list = []

    for x in range(0,n):
        result = ''.join(predict_sequence(objects, coordinates, start_coordinates, c, k, dimension))
        dl = 1 - damerauLevenshtein(sequence, result)
        edit_list.append(dl)
    
    #avg = np.mean(edit_list)
    median = np.median(edit_list)
    return median

In [7]:
def get_avg_editdist(data, dimensions=[[1,'x'],[1,'y'],[1,'z'],[2,'xy'],[2,'xz'],[2,'yz'],[3,'xyz']],n=10,
                    seq='sequence', coords='coordinates', error='error'):
    ''' Calculates average edit distance for all combinations of parameters (c, k, dimension).
        Input: Dataframe with objects, coordinates, start coordinates, object categories
        Output: Dataframe with edit distance results (col name: parameters used)
    '''
    results = pd.DataFrame()
        
    for row in range(0,len(data)):
        objects = list(data.at[row,seq])
        coordinates = {key: ast.literal_eval(value) for key, value in (elem.split(': ') for elem in data.at[row,coords].split(';'))}
        start_coordinates = list(ast.literal_eval(data.at[row, 'start_coordinates']))
        sequence = str(data.at[row,seq])

        try:
            strong_k = list(data.at[row, 'strong_k'].split(','))
        except:
            strong_k = []

        try:
            mid_k = list(data.at[row, 'mid_k'].split(','))
        except:
            mid_k = []

        try:
            food_k = list(data.at[row, 'food_k'].split(','))
        except:
            food_k = []
            
        # set parameters to default values

        c1 = {obj: 1.0 for obj in objects}
        k1 = {obj: 1.0 for obj in objects}
        
        for k2 in np.arange(1.1,2.0,0.1):
            k_food = round(k2,2)
            k1 = {obj: k_food if obj in food_k else 1.0 for obj in objects}
        
            for k in np.arange(0,0.9,0.1):
                k_strong = round(k,2)
                k_mid = round(k + 0.1,2)
                k1 = {obj: k_strong if obj in strong_k else k_mid if obj in mid_k else round(k1[obj],2) for obj in objects}
            
                for c in np.arange(1.0,2.0,0.1):
                    c = round(c, 1)
                    c1 = {obj: c if obj in data.at[row, 'containment'] else 1.0 for obj in objects}
                
                    for dim in dimensions:                
                        # get average edit distance
                        median = get_median(objects, coordinates, start_coordinates, c1, k1, dim, sequence, n)
                        params = 'c: ' + str(c) + '; k: ' + str(k_strong) + ',' + str(k_mid) + ',' + str(k_food) + '; ' + str(dim[1])
                        results.at[row,params] = median
                        
        results.at[row,'sequence'] = sequence
        results.at[row,'error'] = data.at[row,error]
        results.at[row,'ID'] = data.at[row,'ID']
                    
    return results

### Calculate edit distances, create df

In [8]:
# for n=10: 28min
%time results_new = get_avg_editdist(df)

CPU times: user 1h 15min 56s, sys: 1.59 s, total: 1h 15min 57s
Wall time: 1h 15min 55s


In [9]:
results_new 

,"c: 1.0; k: 0.0,0.1,1.1; x","c: 1.0; k: 0.0,0.1,1.1; y","c: 1.0; k: 0.0,0.1,1.1; z","c: 1.0; k: 0.0,0.1,1.1; xy","c: 1.0; k: 0.0,0.1,1.1; xz","c: 1.0; k: 0.0,0.1,1.1; yz","c: 1.0; k: 0.0,0.1,1.1; xyz","c: 1.1; k: 0.0,0.1,1.1; x","c: 1.1; k: 0.0,0.1,1.1; y","c: 1.1; k: 0.0,0.1,1.1; z",...,"c: 1.9; k: 0.8,0.9,1.9; x","c: 1.9; k: 0.8,0.9,1.9; y","c: 1.9; k: 0.8,0.9,1.9; z","c: 1.9; k: 0.8,0.9,1.9; xy","c: 1.9; k: 0.8,0.9,1.9; xz","c: 1.9; k: 0.8,0.9,1.9; yz","c: 1.9; k: 0.8,0.9,1.9; xyz",sequence,error,ID
0,0.666667,0.833333,0.833333,0.833333,0.833333,0.833333,0.833333,0.666667,0.833333,0.833333,...,0.666667,0.833333,0.833333,0.666667,0.833333,0.833333,0.666667,pocgkr,0.723,a1
1,0.750000,0.875000,0.875000,0.875000,0.875000,0.875000,0.875000,0.750000,0.875000,0.875000,...,0.750000,0.750000,0.625000,0.750000,0.625000,0.750000,0.750000,cgwpcfks,0.785,a3
2,0.444444,0.666667,0.555556,0.555556,0.444444,0.666667,0.555556,0.444444,0.666667,0.555556,...,0.444444,0.444444,0.555556,0.333333,0.444444,0.444444,0.333333,kfsfkspwg,0.806,a5
3,0.538462,0.692308,0.692308,0.615385,0.615385,0.692308,0.615385,0.538462,0.692308,0.692308,...,0.538462,0.615385,0.692308,0.538462,0.692308,0.615385,0.538462,pfkswkfsococg,0.862,a11
4,0.888889,0.777778,0.888889,0.666667,0.888889,0.777778,0.666667,0.888889,0.777778,0.888889,...,0.777778,0.888889,0.666667,1.000000,0.777778,0.888889,1.000000,wptgkfsoc,0.806,a13
5,0.600000,0.600000,0.400000,0.400000,0.600000,0.600000,0.400000,0.600000,0.600000,0.400000,...,0.400000,0.800000,0.200000,0.800000,0.400000,0.800000,0.800000,pwfkg,0.666,a16
6,0.833333,0.833333,1.000000,0.500000,0.833333,0.666667,0.500000,0.833333,0.833333,1.000000,...,0.666667,0.833333,0.833333,0.500000,0.666667,0.833333,0.500000,powcer,0.723,a19
7,0.714286,0.428571,0.571429,0.571429,0.714286,0.571429,0.571429,0.714286,0.428571,0.571429,...,0.857143,0.857143,0.571429,0.714286,0.857143,0.857143,0.714286,coprerk,0.756,a22
8,0.900000,0.600000,0.600000,0.600000,0.800000,0.600000,0.600000,0.900000,0.600000,0.600000,...,0.800000,0.900000,0.500000,0.800000,0.700000,0.900000,0.800000,wopofkkskf,0.824,a26
9,0.666667,0.666667,0.666667,0.555556,0.666667,0.666667,0.555556,0.666667,0.666667,0.666667,...,0.666667,0.666667,0.666667,0.555556,0.666667,0.666667,0.555556,opwpwgcoc,0.806,a29


In [58]:
results_new = pd.read_csv('results/results_alldims_n10_2020-10-14.csv', header=0)
results_new = results_new.T
results_new.reset_index(drop=True, inplace=True)

header = results_new.iloc[0]
results_new = results_new[1:]
results_new.columns = header

results_new.drop(results_new.tail(1).index, inplace=True)
results_new

,"c: 1.0; k: 0.0,0.1,1.1; x","c: 1.0; k: 0.0,0.1,1.1; y","c: 1.0; k: 0.0,0.1,1.1; z","c: 1.0; k: 0.0,0.1,1.1; xy","c: 1.0; k: 0.0,0.1,1.1; xz","c: 1.0; k: 0.0,0.1,1.1; yz","c: 1.0; k: 0.0,0.1,1.1; xyz","c: 1.1; k: 0.0,0.1,1.1; x","c: 1.1; k: 0.0,0.1,1.1; y","c: 1.1; k: 0.0,0.1,1.1; z",...,"c: 1.9; k: 0.8,0.9,1.9; x","c: 1.9; k: 0.8,0.9,1.9; y","c: 1.9; k: 0.8,0.9,1.9; z","c: 1.9; k: 0.8,0.9,1.9; xy","c: 1.9; k: 0.8,0.9,1.9; xz","c: 1.9; k: 0.8,0.9,1.9; yz","c: 1.9; k: 0.8,0.9,1.9; xyz",sequence,error,ID
1,0.4,0.8,0.8,0.4,0.2,0.4,0.0,0.2,0.6,0.8,...,0.2,0.3,0.5,0.0,0.3,0.4,0.2,tnpsc,0.6659999999999999,T1
2,0.4,0.6,0.8,0.2,0.3,0.6,0.2,0.2,0.6,0.6,...,0.2,0.6,0.4,0.4,0.4,0.6,0.4,tnpcs,0.6659999999999999,T14
3,0.2,0.6,0.6,0.0,0.2,0.6,0.0,0.3,0.6,0.6,...,0.2,0.4,0.4,0.2,0.3,0.4,0.2,tnpsc,0.6659999999999999,T16
4,0.4,0.4,0.4,0.2,0.4,0.4,0.2,0.4,0.4,0.4,...,0.4,0.2,0.2,0.0,0.0,0.2,0.0,tnspc,0.6659999999999999,T18
5,0.16666666666666666,0.6666666666666666,0.6666666666666666,0.3333333333333333,0.3333333333333333,0.6666666666666666,0.3333333333333333,0.0,0.6666666666666666,0.3333333333333333,...,0.0,0.3333333333333333,0.3333333333333333,0.5,0.0,0.3333333333333333,0.3333333333333333,pms,0.44,E01_01
6,0.7,0.6,0.8,0.8,0.4,0.2,0.4,0.8,0.3,0.8,...,0.7,0.4,0.8,0.4,0.4,0.8,0.8,rhcse,0.6659999999999999,E01_03
7,1.0,0.75,0.875,1.0,1.0,1.0,1.0,0.5,0.5,1.0,...,0.75,0.5,0.5,0.5,0.5,0.5,0.5,pbhs,0.605,E01_05
8,0.5,1.0,0.75,1.0,0.5,1.0,1.0,1.0,1.0,0.625,...,1.0,1.0,0.75,1.0,1.0,1.0,1.0,hscp,0.605,E01_09
9,0.3333333333333333,0.3333333333333333,0.3333333333333333,0.0,0.3333333333333333,0.6666666666666666,0.0,0.0,0.3333333333333333,0.5,...,0.3333333333333333,0.3333333333333333,0.6666666666666666,0.0,0.0,0.6666666666666666,0.0,pbs,0.44,E10_01
10,0.0,0.3333333333333333,0.3333333333333333,0.3333333333333333,0.16666666666666666,0.3333333333333333,0.3333333333333333,0.0,0.3333333333333333,0.0,...,0.3333333333333333,0.6666666666666666,0.3333333333333333,0.3333333333333333,0.3333333333333333,0.3333333333333333,0.3333333333333333,pgs,0.44,E12_01


In [59]:
def check_to_float(x):
    try:
        return float(x)
    except:
        return x
    
for i in results_new.columns:
    results_new[i] = results_new[i].apply(check_to_float)

In [60]:
results_new

,"c: 1.0; k: 0.0,0.1,1.1; x","c: 1.0; k: 0.0,0.1,1.1; y","c: 1.0; k: 0.0,0.1,1.1; z","c: 1.0; k: 0.0,0.1,1.1; xy","c: 1.0; k: 0.0,0.1,1.1; xz","c: 1.0; k: 0.0,0.1,1.1; yz","c: 1.0; k: 0.0,0.1,1.1; xyz","c: 1.1; k: 0.0,0.1,1.1; x","c: 1.1; k: 0.0,0.1,1.1; y","c: 1.1; k: 0.0,0.1,1.1; z",...,"c: 1.9; k: 0.8,0.9,1.9; x","c: 1.9; k: 0.8,0.9,1.9; y","c: 1.9; k: 0.8,0.9,1.9; z","c: 1.9; k: 0.8,0.9,1.9; xy","c: 1.9; k: 0.8,0.9,1.9; xz","c: 1.9; k: 0.8,0.9,1.9; yz","c: 1.9; k: 0.8,0.9,1.9; xyz",sequence,error,ID
1,0.400000,0.800000,0.800000,0.400000,0.200000,0.400000,0.000000,0.200000,0.600000,0.800000,...,0.200000,0.300000,0.500000,0.000000,0.300000,0.400000,0.200000,tnpsc,0.666,T1
2,0.400000,0.600000,0.800000,0.200000,0.300000,0.600000,0.200000,0.200000,0.600000,0.600000,...,0.200000,0.600000,0.400000,0.400000,0.400000,0.600000,0.400000,tnpcs,0.666,T14
3,0.200000,0.600000,0.600000,0.000000,0.200000,0.600000,0.000000,0.300000,0.600000,0.600000,...,0.200000,0.400000,0.400000,0.200000,0.300000,0.400000,0.200000,tnpsc,0.666,T16
4,0.400000,0.400000,0.400000,0.200000,0.400000,0.400000,0.200000,0.400000,0.400000,0.400000,...,0.400000,0.200000,0.200000,0.000000,0.000000,0.200000,0.000000,tnspc,0.666,T18
5,0.166667,0.666667,0.666667,0.333333,0.333333,0.666667,0.333333,0.000000,0.666667,0.333333,...,0.000000,0.333333,0.333333,0.500000,0.000000,0.333333,0.333333,pms,0.440,E01_01
6,0.700000,0.600000,0.800000,0.800000,0.400000,0.200000,0.400000,0.800000,0.300000,0.800000,...,0.700000,0.400000,0.800000,0.400000,0.400000,0.800000,0.800000,rhcse,0.666,E01_03
7,1.000000,0.750000,0.875000,1.000000,1.000000,1.000000,1.000000,0.500000,0.500000,1.000000,...,0.750000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,pbhs,0.605,E01_05
8,0.500000,1.000000,0.750000,1.000000,0.500000,1.000000,1.000000,1.000000,1.000000,0.625000,...,1.000000,1.000000,0.750000,1.000000,1.000000,1.000000,1.000000,hscp,0.605,E01_09
9,0.333333,0.333333,0.333333,0.000000,0.333333,0.666667,0.000000,0.000000,0.333333,0.500000,...,0.333333,0.333333,0.666667,0.000000,0.000000,0.666667,0.000000,pbs,0.440,E10_01
10,0.000000,0.333333,0.333333,0.333333,0.166667,0.333333,0.333333,0.000000,0.333333,0.000000,...,0.333333,0.666667,0.333333,0.333333,0.333333,0.333333,0.333333,pgs,0.440,E12_01


### Get lowest error, compare edit distances

In [14]:
def get_lowest_error(results):
    ''' Returns lowest error in dataframe and index of lowest error.
    '''
    for col in list(results):
        if col != 'sequence' and col != 'error' and col != 'ID':
            results.loc['mean',col] = results[col].mean()
            #results.loc['median',col] = results[col].median()
    lowest = min(results.loc['mean'])
    mean = list(results.loc['mean'])
    
    return lowest, results.columns[(results.loc['mean'] == lowest)], mean, results

In [24]:
lowest, lowest_idx, list_mean, results_mean = get_lowest_error(results_new)
lowest, lowest_idx, np.mean(results_new['error'])

(0.6079403609734492,
 Index(['c: 1.4; k: 0.2,0.3,1.7; xy'], dtype='object'),
 0.7609583333333315)

In [63]:
results_mean

,"c: 1.0; k: 0.0,0.1,1.1; x","c: 1.0; k: 0.0,0.1,1.1; y","c: 1.0; k: 0.0,0.1,1.1; z","c: 1.0; k: 0.0,0.1,1.1; xy","c: 1.0; k: 0.0,0.1,1.1; xz","c: 1.0; k: 0.0,0.1,1.1; yz","c: 1.0; k: 0.0,0.1,1.1; xyz","c: 1.1; k: 0.0,0.1,1.1; x","c: 1.1; k: 0.0,0.1,1.1; y","c: 1.1; k: 0.0,0.1,1.1; z",...,"c: 1.9; k: 0.8,0.9,1.9; x","c: 1.9; k: 0.8,0.9,1.9; y","c: 1.9; k: 0.8,0.9,1.9; z","c: 1.9; k: 0.8,0.9,1.9; xy","c: 1.9; k: 0.8,0.9,1.9; xz","c: 1.9; k: 0.8,0.9,1.9; yz","c: 1.9; k: 0.8,0.9,1.9; xyz",sequence,error,ID
1,0.400000,0.800000,0.800000,0.400000,0.200000,0.400000,0.000000,0.200000,0.600000,0.800000,...,0.200000,0.300000,0.500000,0.000000,0.300000,0.400000,0.200000,tnpsc,0.666,T1
2,0.400000,0.600000,0.800000,0.200000,0.300000,0.600000,0.200000,0.200000,0.600000,0.600000,...,0.200000,0.600000,0.400000,0.400000,0.400000,0.600000,0.400000,tnpcs,0.666,T14
3,0.200000,0.600000,0.600000,0.000000,0.200000,0.600000,0.000000,0.300000,0.600000,0.600000,...,0.200000,0.400000,0.400000,0.200000,0.300000,0.400000,0.200000,tnpsc,0.666,T16
4,0.400000,0.400000,0.400000,0.200000,0.400000,0.400000,0.200000,0.400000,0.400000,0.400000,...,0.400000,0.200000,0.200000,0.000000,0.000000,0.200000,0.000000,tnspc,0.666,T18
5,0.166667,0.666667,0.666667,0.333333,0.333333,0.666667,0.333333,0.000000,0.666667,0.333333,...,0.000000,0.333333,0.333333,0.500000,0.000000,0.333333,0.333333,pms,0.440,E01_01
6,0.700000,0.600000,0.800000,0.800000,0.400000,0.200000,0.400000,0.800000,0.300000,0.800000,...,0.700000,0.400000,0.800000,0.400000,0.400000,0.800000,0.800000,rhcse,0.666,E01_03
7,1.000000,0.750000,0.875000,1.000000,1.000000,1.000000,1.000000,0.500000,0.500000,1.000000,...,0.750000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,pbhs,0.605,E01_05
8,0.500000,1.000000,0.750000,1.000000,0.500000,1.000000,1.000000,1.000000,1.000000,0.625000,...,1.000000,1.000000,0.750000,1.000000,1.000000,1.000000,1.000000,hscp,0.605,E01_09
9,0.333333,0.333333,0.333333,0.000000,0.333333,0.666667,0.000000,0.000000,0.333333,0.500000,...,0.333333,0.333333,0.666667,0.000000,0.000000,0.666667,0.000000,pbs,0.440,E10_01
10,0.000000,0.333333,0.333333,0.333333,0.166667,0.333333,0.333333,0.000000,0.333333,0.000000,...,0.333333,0.666667,0.333333,0.333333,0.333333,0.333333,0.333333,pgs,0.440,E12_01


In [10]:
np.nanmean(list_mean), np.nanmedian(list_mean)

(0.6948012558333564, 0.6958466024130581)

In [16]:
# Compare avg edit distance for xy and xyz
list_xy = []
list_xyz = []

for col in results_mean:
    if col != 'sequence' and col != 'ID' and col != 'error' and col.split(';')[2].strip() == 'xy':
        list_xy.append(results_mean.at['mean',col])
    elif col != 'sequence' and col != 'ID' and col != 'error' and col.split(';')[2].strip() == 'xyz':
        list_xyz.append(results_mean.at['mean',col])
        
avg_xy = np.mean(list_xy)
med_xy = np.median(list_xy)
std_xy = np.std(list_xy)
print('Average xy: ' + str(avg_xy)  + ', stdev: ' + str(std_xy) + ', median: ' + str(med_xy))
        
avg_xyz = np.mean(list_xyz)
med_xyz = np.median(list_xyz)
std_xyz = np.std(list_xyz)
print('Average xyz: ' + str(avg_xyz) + ', stdev: ' + str(std_xyz) + ', median: ' + str(med_xyz))

Average xy: 0.6470686936000863, stdev: 0.029724013341797736, median: 0.6398675070747024
Average xyz: 0.6453980619707905, stdev: 0.02942956701153299, median: 0.6346879065826019


In [17]:
# Compare avg edit distances for x, y, z
list_x = []
list_y = []
list_z = []
list_xz = []
list_yz = []

for col in results_mean:
    if col != 'sequence' and col != 'ID' and col != 'error' and col.split(';')[2].strip() == 'x':
        list_x.append(results_mean.at['mean',col])
    elif col != 'sequence' and col != 'ID' and col != 'error' and col.split(';')[2].strip() == 'y':
        list_y.append(results_mean.at['mean',col])
    elif col != 'sequence' and col != 'ID' and col != 'error' and col.split(';')[2].strip() == 'z':
        list_z.append(results_mean.at['mean',col])
    elif col != 'sequence' and col != 'ID' and col != 'error' and col.split(';')[2].strip() == 'xz':
        list_xz.append(results_mean.at['mean',col])
    elif col != 'sequence' and col != 'ID' and col != 'error' and col.split(';')[2].strip() == 'yz':
        list_yz.append(results_mean.at['mean',col])

avg_xz = np.mean(list_xz)
avg_yz = np.mean(list_yz)
        
avg_x = np.mean(list_x)
lowest_x = min(list_x)
print('Average x: ' + str(avg_x) + ', min: ' + str(lowest_x)) 
        
avg_y = np.mean(list_y)
lowest_y = min(list_y)
print('Average y: ' + str(avg_y) + ', min: ' + str(lowest_y)) 
        
avg_z = np.mean(list_z)
lowest_z = min(list_z)
print('Average z: ' + str(avg_z) + ', min: ' + str(lowest_z)) 

print('Average xz: ' + str(avg_xz)) 
print('Average yz: ' + str(avg_yz)) 

Average x: 0.7134384679150084, min: 0.6919793709053161
Average y: 0.6953444855794197, min: 0.6589018746581772
Average z: 0.7179791484956861, min: 0.6872032335661535
Average xz: 0.7035784117916113
Average yz: 0.694544926581072


### Statistical analysis

In [18]:
# Compare all error predictions for dimensions
stat, p = friedmanchisquare(list_x,list_y,list_z,list_xy,list_xz,list_yz,list_xyz)
print('Friedman: stat = %.3f, p = % 10.3E' % (stat, p))

Friedman: stat = 3023.356, p =  0.000E+00


In [20]:
# 2D vs 3D median
stat, p = wilcoxon(list_xy, list_xyz, zero_method='wilcox')
print('Wilcoxon: W = %.3f, p = %.5f' % (stat, p))

Wilcoxon: W = 91685.000, p = 0.00000


### Plots for dataframe

In [21]:
# Define input for plots

c = [float(x[3:6]) for x in results_new.columns.tolist() if x != 'sequence' and x != 'ID' and x != 'error']
k = [float(x[11:14]) for x in results_new.columns.tolist() if x != 'sequence' and x != 'ID' and x != 'error']
dim = [x.strip() for x in results_new.columns[:-3].str.split(';').str[2]]
median = [x for x in results_mean.loc['mean'].tolist() if pd.notnull(x)]

In [22]:
%matplotlib qt

cm = ['red','blue','green','magenta','cyan','orange','grey']
cm = ['#e74c3c', '#0064a3', '#70b85d', '#287d78', '#54d0ff', '#f1e664', '#fd8f00']
dim_num = [0 if x=='x' else 1 if x=='y' else 2 if x=='z' else 3 if x=='xy' else 4 if x=='xz' else 5 if x=='yz' else 6 for x in dim]

cmap = matplotlib.colors.ListedColormap(cm)

ticks = ['x', 'y', 'z', 'xy', 'xz', 'yz', 'xyz']
norm = matplotlib.colors.BoundaryNorm(ticks, cmap.N)

# create figure, 3d grid, set background to white
fig2 = plt.figure(figsize=(12,8))
ax2 = fig2.add_subplot(111, projection='3d')
ax2.w_xaxis.set_pane_color((1.0,1.0,1.0,1.0))
ax2.w_yaxis.set_pane_color((1.0,1.0,1.0,1.0))
ax2.w_zaxis.set_pane_color((1.0,1.0,1.0,1.0))

# create plot w/ median edit distance
img = ax2.scatter(c, k, median, alpha=0.5, s=38, c=dim_num, cmap=cmap)

# plot horizontal plane for baseline
xx, yy = np.meshgrid(np.linspace(1,1.9), np.linspace(0,0.9))
#yy = np.meshgrid(range(2), range(2))
zz = xx * 0 + np.mean(results_mean['error'])
ax2.plot_surface(xx, yy, zz, alpha=0.5)

# set labels
ax2.set_ylabel('parameter k', fontsize=20, labelpad=10)
ax2.set_xlabel('parameter c', fontsize=20, labelpad=10)
ax2.set_zlabel('normalized edit distance', fontsize=20, labelpad=7)
#plt.title('Average edit distance (1 step)', fontsize=24)

# create colorbar
cb = plt.colorbar(img, cax = fig2.add_axes([0.82,0.3,0.03,0.4]))
cb.set_alpha(0.8)
cb.draw_all()
cb.ax.set_yticklabels(ticks, fontsize=16)
plt.show()

/home/zauri/files/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:38: UserWarning: FixedFormatter should only be used together with FixedLocator


In [33]:
distances = [0.33333333333333337, 0.5, 0.6666666666666667, 0.6153846153846154, 0.5555555555555556, 0.19999999999999996, 0.5, 0.4285714285714286, 0.6, 0.4444444444444444, 0.6, 0.625, 0.5384615384615384, 0.5, 0.33333333333333337, 0.5, 0.7272727272727273, 0.5, 0.5, 0.6666666666666667, 0.6363636363636364, 0.625, 0.6666666666666667, 0.5882352941176471, 0.125, 0.375, 0.5833333333333333, 0.5555555555555556, 0.5714285714285714, 0.19999999999999996, 0.25, 0.6923076923076923, 0.4444444444444444, 0.7, 0.4285714285714286, 0.5, 0.7272727272727273, 0.625, 0.6363636363636364, 0.33333333333333337, 0.5, 0.4, 0.6, 0.2857142857142857, 0.6, 0.25, 0.4, 0.5555555555555556, 0.5, 0.375, 0.0, 0.4, 0.6666666666666667, 0.4285714285714286, 0.625, 0.6363636363636364, 0.5555555555555556, 0.5714285714285714, 0.6, 0.4, 0.25, 0.6923076923076923, 0.7777777777777778, 0.4285714285714286, 0.5, 0.19999999999999996, 0.46153846153846156, 0.5384615384615384, 0.5, 0.5714285714285714, 0.5, 0.25, 0.6666666666666667, 0.7, 0.33333333333333337, 0.7692307692307692, 0.6363636363636364, 0.5714285714285714, 0.6363636363636364, 0.375, 0.19999999999999996, 0.0, 0.19999999999999996, 0.19999999999999996, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.4, 0.33333333333333337, 0.25, 0.2857142857142857, 0.16666666666666663, 0.7, 0.0, 0.8666666666666667, 0.5, 0.4, 0.25, 0.5, 0.375, 0.375, 0.5, 0.5, 0.5, 0.375, 0.25, 0.375, 0.375, 0.25, 0.25, 0.375, 0.25, 0.25, 0.375, 0.5, 0.625, 0.25, 0.375, 0.375, 0.5, 0.5, 0.5, 0.25, 0.375, 0.5, 0.625, 0.0, 0.5, 0.5, 0.5, 0.375, 0.375, 0.625, 0.5, 0.375, 0.5, 0.5, 0.25, 0.125, 0.5, 0.5, 0.375, 0.5, 0.125, 0.375, 0.5, 0.5, 0.5, 0.5, 0.125, 0.125, 0.0, 0.375, 0.5, 0.375, 0.375, 0.375, 0.0, 0.125, 0.0, 0.25, 0.375, 0.5, 0.5, 0.625]

### plot best model

In [38]:
#%matplotlib inline
%matplotlib qt

IDs = df['ID']

error = df['error']

seqs= results_mean['sequence'][:-1].values

res = results_new['c: 1.4; k: 0.2,0.3,1.7; xy'][:-1].values

x = [x for x in range (0,len(seqs))]

# plot scatter + lines for all simulations
plt.scatter(x, res, marker='o', s=20, c='blue', alpha=0.6, 
            label=str('model-generated avg.: ') + str(round(lowest,2)))
plt.plot(x, res, c='blue', alpha=0.6)

# plot CPT baseline
plt.scatter(x, distances, marker='o', s=20, c='darkgreen', alpha=0.3, 
            label=str('CPT baseline avg.: ') + str(round(np.mean(distances),2)))
plt.plot(x, distances, '-', c='darkgreen', alpha=0.3)
#plt.plot((x_original,x_original),(res_original,distances), '--', c='darkgreen', alpha=0.6)
plt.fill_between(x, distances, alpha=0.2, color='darkgreen')

# plot random baseline
plt.scatter(x, error, c='red', s=20, marker='o', alpha=0.3, 
            label=str('guessing baseline avg.: ') + str(round(np.mean(results_new['error']),2)))
plt.plot(x, error, c='red', alpha=0.3)#, 
         #label=str('guessing baseline avg.: ' + str(round(np.mean(results_original['error']),2))))
plt.fill_between(x, error, [1.0 * len(error)], color='red', alpha=0.2)


plt.xticks(x, labels=IDs, rotation=90, fontsize=5)
#plt.xticklabels(IDs, rotation=90, fontsize=6)

plt.ylabel('edit distance', fontsize=10)
plt.xlabel('sequence', fontsize=10)
plt.ylim(0.0, 1.0)
plt.title('best model (all), c:1.4; k: 0.2,0.3,1.7; xy')

plt.legend(fontsize=8, framealpha=0.9, loc='lower left')

#plt.savefig('plot_median_editdist_individualerrors_diff.png', bbox_inches='tight')
plt.show()

In [32]:
len(IDs)

168